In [1]:
###importaciones
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import category_encoders
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder 
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz
import graphviz
import joblib

In [2]:
##Importamos el excel con los datos para la certificación y los pkl que creamos.
data=pd.read_excel('Cert_Caliente_19_01_21.xlsx')
dt=joblib.load('ModeloCaliente.pkl')
enc=joblib.load('TargetCaliente.pkl')
encobj=joblib.load('EncodificacionObjetivo.pkl') ##éste es único ya que solo cumple función de traductor al final.
data

,idMaterial,ACEROOrigen,CPRODTOOrigen,CCLASEPRODTOOrigen,N_ANCHO,N_ESPESOR,Clasificacion_Norma,C_ACEROOrigen,BORDESOrigen,DIAM__INTERNOOrigen,...,BORDESDestino,DIAM__INTERNODestino,EXTREMOSDestino,NORMADestino,NORTOLANCHODestino,NORTOLESPESORDestino,PLANITUDDestino,SOLDADestino,SUPERFICIEDestino,USOGRALDestino
0,2A849644GM200,7092,MP501073,M-P-CALIENTE-BOB-_,0.0,0.000,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
1,2A850796GM200,7061,MP501073,M-P-CALIENTE-BOB-_,999.0,0.800,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA \ TUBE GLV (V 2015 REV 0),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
2,2A851998GM200,7094,MP509656,M-P-CALIENTE-BOB-_,1206.0,0.770,HSLA,HS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
3,2A854123GM200,7061,MP501073,M-P-CALIENTE-BOB-_,149.5,3.150,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
4,2A857872GM200,2060,MP501073,M-P-CALIENTE-BOB-_,97.4,1.825,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2A879262GM200,7092,MP501073,M-P-CALIENTE-BOB-_,NaN,NaN,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
123,2A879254GM200,4092,MP501073,M-P-CALIENTE-BOB-_,NaN,NaN,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
124,2A884998GM200,2060,MP501073,M-P-CALIENTE-BOB-_,NaN,NaN,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
125,2A883720GM200,2084,MP509656,M-P-CALIENTE-BOB-_,NaN,NaN,HSLA,HS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA \ TUBE GLV (V 2015 REV 0),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES


In [3]:
data=pd.DataFrame(data)
data

,idMaterial,ACEROOrigen,CPRODTOOrigen,CCLASEPRODTOOrigen,N_ANCHO,N_ESPESOR,Clasificacion_Norma,C_ACEROOrigen,BORDESOrigen,DIAM__INTERNOOrigen,...,BORDESDestino,DIAM__INTERNODestino,EXTREMOSDestino,NORMADestino,NORTOLANCHODestino,NORTOLESPESORDestino,PLANITUDDestino,SOLDADestino,SUPERFICIEDestino,USOGRALDestino
0,2A849644GM200,7092,MP501073,M-P-CALIENTE-BOB-_,0.0,0.000,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
1,2A850796GM200,7061,MP501073,M-P-CALIENTE-BOB-_,999.0,0.800,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA \ TUBE GLV (V 2015 REV 0),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
2,2A851998GM200,7094,MP509656,M-P-CALIENTE-BOB-_,1206.0,0.770,HSLA,HS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
3,2A854123GM200,7061,MP501073,M-P-CALIENTE-BOB-_,149.5,3.150,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
4,2A857872GM200,2060,MP501073,M-P-CALIENTE-BOB-_,97.4,1.825,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2A879262GM200,7092,MP501073,M-P-CALIENTE-BOB-_,NaN,NaN,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
123,2A879254GM200,4092,MP501073,M-P-CALIENTE-BOB-_,NaN,NaN,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
124,2A884998GM200,2060,MP501073,M-P-CALIENTE-BOB-_,NaN,NaN,CS,CS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
125,2A883720GM200,2084,MP509656,M-P-CALIENTE-BOB-_,NaN,NaN,HSLA,HS,LAMINACIÓN,711,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA \ TUBE GLV (V 2015 REV 0),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES


In [4]:
##Limpiamos los datos recogiendo solo las columnas que necesitamos para la predicción
databuena=data.loc[:, ['ACEROOrigen', 'ACERODestino', 'BORDESOrigen', 'DIAM__INTERNOOrigen',
       'EXTREMOSOrigen', 'NORMAOrigen', 'NORTOLANCHOOrigen',
       'NORTOLESPESOROrigen', 'PLANITUDOrigen', 'SOLDAOrigen',
       'SUPERFICIEOrigen', 'USOGRALOrigen', 'BORDESDestino',
       'DIAM__INTERNODestino', 'EXTREMOSDestino', 'NORMADestino',
       'NORTOLANCHODestino', 'NORTOLESPESORDestino', 'PLANITUDDestino',
       'SOLDADestino', 'SUPERFICIEDestino', 'USOGRALDestino']] 
databuena

,ACEROOrigen,ACERODestino,BORDESOrigen,DIAM__INTERNOOrigen,EXTREMOSOrigen,NORMAOrigen,NORTOLANCHOOrigen,NORTOLESPESOROrigen,PLANITUDOrigen,SOLDAOrigen,...,BORDESDestino,DIAM__INTERNODestino,EXTREMOSDestino,NORMADestino,NORTOLANCHODestino,NORTOLESPESORDestino,PLANITUDDestino,SOLDADestino,SUPERFICIEDestino,USOGRALDestino
0,7092,NaN,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,ASTM A 569 \ TIPO B (V 1997 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
1,7061,7092,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,ASTM A 569 \ TIPO B (V 1997 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA \ TUBE GLV (V 2015 REV 0),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
2,7094,7092,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,ASTM A 1011 \ HSLAS 45 CLASE 2 (V 2006 REV 2),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
3,7061,C091TM,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,ASTM A 569 \ TIPO B (V 1997 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
4,2060,NaN,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,ASTM A 569 \ TIPO B (V 1997 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,7092,C091TM,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,ASTM A 569 \ TIPO B (V 1997 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
123,4092,C091TM,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,ASTM A 569 \ TIPO B (V 1997 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
124,2060,7092,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,ASTM A 569 \ TIPO B (V 1997 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES
125,2084,7092,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,ASTM A 1011 \ HSLAS 45 CLASE 2 (V 2006 REV 2),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,...,LAMINACIÓN,711,NO CORTADOS 98% EN ESPESOR.,TER \ PER-GA \ TUBE GLV (V 2015 REV 0),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES


In [5]:
databuena.columns

Index(['ACEROOrigen', 'ACERODestino', 'BORDESOrigen', 'DIAM__INTERNOOrigen',
       'EXTREMOSOrigen', 'NORMAOrigen', 'NORTOLANCHOOrigen',
       'NORTOLESPESOROrigen', 'PLANITUDOrigen', 'SOLDAOrigen',
       'SUPERFICIEOrigen', 'USOGRALOrigen', 'BORDESDestino',
       'DIAM__INTERNODestino', 'EXTREMOSDestino', 'NORMADestino',
       'NORTOLANCHODestino', 'NORTOLESPESORDestino', 'PLANITUDDestino',
       'SOLDADestino', 'SUPERFICIEDestino', 'USOGRALDestino'],
      dtype='object')

In [6]:
##Usamos el PKL del encodificacdor para transofrmar la información
datae=enc.transform(databuena)
datae

,ACEROOrigen,ACERODestino,BORDESOrigen,DIAM__INTERNOOrigen,EXTREMOSOrigen,NORMAOrigen,NORTOLANCHOOrigen,NORTOLESPESOROrigen,PLANITUDOrigen,SOLDAOrigen,...,BORDESDestino,DIAM__INTERNODestino,EXTREMOSDestino,NORMADestino,NORTOLANCHODestino,NORTOLESPESORDestino,PLANITUDDestino,SOLDADestino,SUPERFICIEDestino,USOGRALDestino
0,0.505072,0.505072,0.505072,711,0.505072,0.690153,0.511936,0.599888,0.505072,0.505072,...,0.505072,711,0.505072,0.701543,0.553336,0.568254,0.505072,0.505072,0.557074,0.643462
1,0.505072,0.490260,0.505072,711,0.505072,0.690153,0.511936,0.599888,0.505072,0.505072,...,0.505072,711,0.505072,0.886700,0.553336,0.463054,0.505072,0.505072,0.557074,0.643462
2,0.505072,0.490260,0.505072,711,0.505072,0.661885,0.511936,0.599888,0.505072,0.505072,...,0.505072,711,0.505072,0.701543,0.553336,0.463054,0.505072,0.505072,0.557074,0.643462
3,0.505072,0.873698,0.505072,711,0.505072,0.690153,0.511936,0.599888,0.505072,0.505072,...,0.505072,711,0.505072,0.701543,0.553336,0.463054,0.505072,0.505072,0.557074,0.643462
4,0.505072,0.505072,0.505072,711,0.505072,0.690153,0.511936,0.599888,0.505072,0.505072,...,0.505072,711,0.505072,0.701543,0.553336,0.463054,0.505072,0.505072,0.557074,0.643462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0.505072,0.873698,0.505072,711,0.505072,0.690153,0.511936,0.599888,0.505072,0.505072,...,0.505072,711,0.505072,0.701543,0.553336,0.463054,0.505072,0.505072,0.557074,0.643462
123,0.505072,0.873698,0.505072,711,0.505072,0.690153,0.511936,0.599888,0.505072,0.505072,...,0.505072,711,0.505072,0.701543,0.553336,0.463054,0.505072,0.505072,0.557074,0.643462
124,0.505072,0.490260,0.505072,711,0.505072,0.690153,0.511936,0.599888,0.505072,0.505072,...,0.505072,711,0.505072,0.701543,0.553336,0.463054,0.505072,0.505072,0.557074,0.643462
125,0.505072,0.490260,0.505072,711,0.505072,0.661885,0.511936,0.599888,0.505072,0.505072,...,0.505072,711,0.505072,0.886700,0.553336,0.463054,0.505072,0.505072,0.557074,0.643462


In [7]:
##Se hace las predicciones tanto normales como de probabilidad.
predicprob=dt.predict_proba(datae)
predic=dt.predict(datae)

In [8]:
##Se crean 2 DF que contengan la información de ambas predicciones.
ResultadosProb=pd.DataFrame(data=predicprob,columns=['Prob. Ok','Prob. Rechazado'])
ResultadosNormal=pd.DataFrame(data=predic,columns=['Reasignación'])
ResultadosNormal=pd.DataFrame(data=encobj.inverse_transform(ResultadosNormal),columns=['Reasignacion'])
ResultadosNormal

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,Reasignacion
0,Rechazado
1,OK
2,Rechazado
3,Rechazado
4,OK
...,...
122,Rechazado
123,Rechazado
124,OK
125,Rechazado


In [9]:
ResultadosProb

,Prob. Ok,Prob. Rechazado
0,0.000000,1.000000
1,0.769231,0.230769
2,0.000000,1.000000
3,0.000000,1.000000
4,0.539007,0.460993
...,...,...
122,0.000000,1.000000
123,0.000000,1.000000
124,0.539007,0.460993
125,0.000000,1.000000


In [10]:
##Pegamos la data original con las dos predicciones.
concatenated = pd.concat([data,ResultadosNormal, ResultadosProb],axis=1) 
concatenated

,idMaterial,ACEROOrigen,CPRODTOOrigen,CCLASEPRODTOOrigen,N_ANCHO,N_ESPESOR,Clasificacion_Norma,C_ACEROOrigen,BORDESOrigen,DIAM__INTERNOOrigen,...,NORMADestino,NORTOLANCHODestino,NORTOLESPESORDestino,PLANITUDDestino,SOLDADestino,SUPERFICIEDestino,USOGRALDestino,Reasignacion,Prob. Ok,Prob. Rechazado
0,2A849644GM200,7092,MP501073,M-P-CALIENTE-BOB-_,0.0,0.000,CS,CS,LAMINACIÓN,711,...,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES,Rechazado,0.000000,1.000000
1,2A850796GM200,7061,MP501073,M-P-CALIENTE-BOB-_,999.0,0.800,CS,CS,LAMINACIÓN,711,...,TER \ PER-GA \ TUBE GLV (V 2015 REV 0),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES,OK,0.769231,0.230769
2,2A851998GM200,7094,MP509656,M-P-CALIENTE-BOB-_,1206.0,0.770,HSLA,HS,LAMINACIÓN,711,...,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES,Rechazado,0.000000,1.000000
3,2A854123GM200,7061,MP501073,M-P-CALIENTE-BOB-_,149.5,3.150,CS,CS,LAMINACIÓN,711,...,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES,Rechazado,0.000000,1.000000
4,2A857872GM200,2060,MP501073,M-P-CALIENTE-BOB-_,97.4,1.825,CS,CS,LAMINACIÓN,711,...,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES,OK,0.539007,0.460993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2A879262GM200,7092,MP501073,M-P-CALIENTE-BOB-_,NaN,NaN,CS,CS,LAMINACIÓN,711,...,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES,Rechazado,0.000000,1.000000
123,2A879254GM200,4092,MP501073,M-P-CALIENTE-BOB-_,NaN,NaN,CS,CS,LAMINACIÓN,711,...,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES,Rechazado,0.000000,1.000000
124,2A884998GM200,2060,MP501073,M-P-CALIENTE-BOB-_,NaN,NaN,CS,CS,LAMINACIÓN,711,...,TER \ PER-GA (V 2006 REV 1),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES,OK,0.539007,0.460993
125,2A883720GM200,2084,MP509656,M-P-CALIENTE-BOB-_,NaN,NaN,HSLA,HS,LAMINACIÓN,711,...,TER \ PER-GA \ TUBE GLV (V 2015 REV 0),ASTM A 568 / A 635,1/2 ASTM A 568 / A 635,TERNIUM ESTANDAR,NO,INTERNA,TUBOS Y PERFILES,Rechazado,0.000000,1.000000


In [12]:
##Se crea un archivo de excel 
concatenated.to_excel("ResultadoCaliente2021Final.xlsx")  


In [11]:
## Se hace una verificación rápida de cuantos rechazos y aceptados hubo.
concatenated.groupby('Reasignacion').size()

Reasignacion
OK            24
Rechazado    103
dtype: int64